In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from selenium import webdriver
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
import re
import time
from textblob import TextBlob

start_time = time.time()
ret = 1
page_num = 1
rating = []
name = []
review_date = []
title = []
review = []
all_data = dict()
url = Embassy_Suites
driver = webdriver.Chrome()   
    
while ret:
    print(url)
    
    driver.get(url)
#     wait = ui.WebDriverWait(driver,10)
    
    mores = driver.find_elements_by_css_selector('span.taLnk.ulBlueLinks')
    for more in mores:
        driver.execute_script("arguments[0].click()", more) 

    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')   
    
    data = dict()

    for mgr in soup.find_all("div", {'class':'mgrRspnInline'}): 
        mgr.decompose()
    for cnt in soup.find_all("div", {'class':'content_block answers_block tabs_answers scroll_tabs'}):
        cnt.decompose()    

    for sp in soup.find_all("div", {'class':'rating reviewItemInline'}):
        sp = str(sp.find("span")['class'][1])[7]
        rating.append(sp)

    for uinfo in soup.find_all("div", {'class':'username mo'}):
        name.append(uinfo.getText())

    for rv in soup.find_all("p", {'class':'partial_entry'}): 
        review.append(rv.getText())

    for tt in soup.find_all("span", {'class':'noQuotes'}):
        title.append(tt.getText())

    for rvdt in soup.find_all("span", {'class':'ratingDate relativeDate'}):
        review_date.append(rvdt['title']) 

    page_number = int(soup.find('span', class_="pageNum current")['data-page-number'])

    data['title'] = title
    data['review'] = review
    data['review_date'] = review_date
    data['rating'] = rating
    data['name'] = name
    

    all_data.update(data)  
    all_data['review'] = [s.strip('\n') for s in all_data['review']]
    all_data['name'] = [s.strip('\n') for s in all_data['name']]
    
    print(page_num)
    page_num += 1
    
    next_page = soup.find('link', rel="next")
    if next_page:
        next_page_url = next_page['href']
        
    next_url = 'http://www.tripadvisor.com' + next_page_url    
    
    if next_page:
        url = 'http://www.tripadvisor.com' + next_page_url
    else:
        ret = 0

review_df = pd.DataFrame.from_dict(all_data, orient='index')
review_df = review_df.transpose()
review_df = review_df[['review_date','name','rating','title','review']]
review_df.to_excel('Embassy_Suites.xlsx')      
       
print("---took %s seconds ---" % (time.time() - start_time))
print("Data Imported")